# ERDDCI with Prompt-to-Prompt(Stable Diffusion)


In [1]:
from typing import Optional, Union, Tuple, List, Callable, Dict
import os, gc
import abc
import torch
from tqdm.notebook import tqdm
from diffusers import StableDiffusionPipeline, DDIMScheduler
import torch.nn.functional as nnf
import numpy as np
import pandas as pd
from PIL import Image
import lpips
from skimage.metrics import structural_similarity as SSIM
from skimage.metrics import peak_signal_noise_ratio as PSNR

import ptp_utils
import seq_aligner

For loading the Stable Diffusion using Diffusers, follow the instuctions https://huggingface.co/blog/stable_diffusion and update `MY_TOKEN` with your token.
Set `LOW_RESOURCE` to `True` for running on 12GB GPU.


In [3]:
# MY_TOKEN = ''
LOW_RESOURCE = False
NUM_DIFFUSION_STEPS = 50
GUIDANCE_SCALE = 1.0
MAX_NUM_WORDS = 77 * 4
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
gc.collect()
torch.cuda.empty_cache()

In [5]:
scheduler = DDIMScheduler(
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
)
ldm_stable = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", local_files_only=True, scheduler=scheduler
).to(device)
try:
    ldm_stable.disable_xformers_memory_efficient_attention()
except AttributeError:
    print("Attribute disable_xformers_memory_efficient_attention() is missing")
tokenizer = ldm_stable.tokenizer

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/home/djm/ERDDCI/.venv/lib/python3.11/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:223: FutureWarning: The configuration file of this scheduler: DDIMScheduler {
  "_class_name": "DDIMScheduler",
  "_diffusers_version": "0.33.1",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "clip_sample_range": 1.0,
  "dynamic_thresholding_ratio": 0.995,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "rescale_betas_zero_snr": false,
  "sample_max_value": 1.0,
  "set_alpha_to_one": false,
  "steps_offset": 0,
  "thresholding": false,
  "timestep_spacing": "leading",
  "trained_betas": null
}
 is outdated. `steps_offset` should be set to 1 instead of 0. Please make sure to update the config accordingly as leaving `steps_offset` might led to incorrect results in future versions. If you have downloaded this checkpoint from the Hugging Face Hub, it would be very nice if you could open a Pull re

### Prompt-to-Prompt Attnetion Controllers

The warnings and errors in this P2P code section are non-critical - please proceed with execution.


In [6]:
class LocalBlend:
    def __call__(self, x_t, attention_store):
        k = 1
        maps = attention_store["down_cross"][2:4] + attention_store["up_cross"][:3]
        maps = [
            item.reshape(self.alpha_layers.shape[0], -1, 1, 16, 16, MAX_NUM_WORDS)
            for item in maps
        ]
        maps = torch.cat(maps, dim=1)
        maps = (maps * self.alpha_layers).sum(-1).mean(1)
        mask = nnf.max_pool2d(maps, (k * 2 + 1, k * 2 + 1), (1, 1), padding=(k, k))
        mask = nnf.interpolate(mask, size=(x_t.shape[2:]))
        mask = mask / mask.max(2, keepdims=True)[0].max(3, keepdims=True)[0]
        mask = mask.gt(self.threshold)
        mask = (mask[:1] + mask[1:]).float()
        x_t = x_t[:1] + mask * (x_t - x_t[:1])
        return x_t

    def __init__(self, prompts: List[str], words: [List[List[str]]], threshold=0.3):
        alpha_layers = torch.zeros(len(prompts), 1, 1, 1, 1, MAX_NUM_WORDS)
        for i, (prompt, words_) in enumerate(zip(prompts, words)):
            if type(words_) is str:
                words_ = [words_]
            for word in words_:
                ind = ptp_utils.get_word_inds(prompt, word, tokenizer)
                alpha_layers[i, :, :, :, :, ind] = 1
        self.alpha_layers = alpha_layers.to(device)
        self.threshold = threshold


class AttentionControl(abc.ABC):
    def step_callback(self, x_t):
        return x_t

    def between_steps(self):
        return

    @property
    def num_uncond_att_layers(self):
        return self.num_att_layers if LOW_RESOURCE else 0

    @abc.abstractmethod
    def forward(self, attn, is_cross: bool, place_in_unet: str):
        raise NotImplementedError

    def __call__(self, attn, is_cross: bool, place_in_unet: str):
        if self.cur_att_layer >= self.num_uncond_att_layers:
            if LOW_RESOURCE:
                attn = self.forward(attn, is_cross, place_in_unet)
            else:
                h = attn.shape[0]
                attn[h // 2 :] = self.forward(attn[h // 2 :], is_cross, place_in_unet)
        self.cur_att_layer += 1
        if self.cur_att_layer == self.num_att_layers + self.num_uncond_att_layers:
            self.cur_att_layer = 0
            self.cur_step += 1
            self.between_steps()
        return attn

    def reset(self):
        self.cur_step = 0
        self.cur_att_layer = 0

    def __init__(self):
        self.cur_step = 0
        self.num_att_layers = -1
        self.cur_att_layer = 0


class EmptyControl(AttentionControl):
    def forward(self, attn, is_cross: bool, place_in_unet: str):
        return attn


class AttentionStore(AttentionControl):
    @staticmethod
    def get_empty_store():
        return {
            "down_cross": [],
            "mid_cross": [],
            "up_cross": [],
            "down_self": [],
            "mid_self": [],
            "up_self": [],
        }

    def forward(self, attn, is_cross: bool, place_in_unet: str):
        key = f"{place_in_unet}_{'cross' if is_cross else 'self'}"
        if attn.shape[1] <= 32**2:  # avoid memory overhead
            self.step_store[key].append(attn)
        return attn

    def between_steps(self):
        if len(self.attention_store) == 0:
            self.attention_store = self.step_store
        else:
            for key in self.attention_store:
                for i in range(len(self.attention_store[key])):
                    self.attention_store[key][i] += self.step_store[key][i]
        self.step_store = self.get_empty_store()

    def get_average_attention(self):
        average_attention = {
            key: [item / self.cur_step for item in self.attention_store[key]]
            for key in self.attention_store
        }
        return average_attention

    def reset(self):
        super(AttentionStore, self).reset()
        self.step_store = self.get_empty_store()
        self.attention_store = {}

    def __init__(self):
        super(AttentionStore, self).__init__()
        self.step_store = self.get_empty_store()
        self.attention_store = {}


class AttentionControlEdit(AttentionStore, abc.ABC):
    def step_callback(self, x_t):
        if self.local_blend is not None:
            x_t = self.local_blend(x_t, self.attention_store)
        return x_t

    def replace_self_attention(self, attn_base, att_replace):
        if att_replace.shape[2] <= 16**2:
            return attn_base.unsqueeze(0).expand(att_replace.shape[0], *attn_base.shape)
        else:
            return att_replace

    @abc.abstractmethod
    def replace_cross_attention(self, attn_base, att_replace):
        raise NotImplementedError

    def forward(self, attn, is_cross: bool, place_in_unet: str):
        super(AttentionControlEdit, self).forward(attn, is_cross, place_in_unet)
        if is_cross or (
            self.num_self_replace[0] <= self.cur_step < self.num_self_replace[1]
        ):
            h = attn.shape[0] // (self.batch_size)
            attn = attn.reshape(self.batch_size, h, *attn.shape[1:])
            attn_base, attn_repalce = attn[0], attn[1:]
            if is_cross:
                alpha_words = self.cross_replace_alpha[self.cur_step]
                attn_repalce_new = (
                    self.replace_cross_attention(attn_base, attn_repalce) * alpha_words
                    + (1 - alpha_words) * attn_repalce
                )
                attn[1:] = attn_repalce_new
            else:
                attn[1:] = self.replace_self_attention(attn_base, attn_repalce)
            attn = attn.reshape(self.batch_size * h, *attn.shape[2:])
        return attn

    def __init__(
        self,
        prompts,
        num_steps: int,
        cross_replace_steps: Union[
            float, Tuple[float, float], Dict[str, Tuple[float, float]]
        ],
        self_replace_steps: Union[float, Tuple[float, float]],
        local_blend: Optional[LocalBlend],
    ):
        super(AttentionControlEdit, self).__init__()
        self.batch_size = len(prompts)
        self.cross_replace_alpha = ptp_utils.get_time_words_attention_alpha(
            prompts, num_steps, cross_replace_steps, tokenizer
        ).to(device)
        if type(self_replace_steps) is float:
            self_replace_steps = 0, self_replace_steps
        self.num_self_replace = (
            int(num_steps * self_replace_steps[0]),
            int(num_steps * self_replace_steps[1]),
        )
        self.local_blend = local_blend


class AttentionReplace(AttentionControlEdit):
    def replace_cross_attention(self, attn_base, att_replace):
        return torch.einsum("hpw,bwn->bhpn", attn_base, self.mapper)

    def __init__(
        self,
        prompts,
        num_steps: int,
        cross_replace_steps: float,
        self_replace_steps: float,
        local_blend: Optional[LocalBlend] = None,
    ):
        super(AttentionReplace, self).__init__(
            prompts, num_steps, cross_replace_steps, self_replace_steps, local_blend
        )
        self.mapper = seq_aligner.get_replacement_mapper(prompts, tokenizer).to(device)


class AttentionRefine(AttentionControlEdit):
    def replace_cross_attention(self, attn_base, att_replace):
        attn_base_replace = attn_base[:, :, self.mapper].permute(2, 0, 1, 3)
        attn_replace = attn_base_replace * self.alphas + att_replace * (1 - self.alphas)
        return attn_replace

    def __init__(
        self,
        prompts,
        num_steps: int,
        cross_replace_steps: float,
        self_replace_steps: float,
        local_blend: Optional[LocalBlend] = None,
    ):
        super(AttentionRefine, self).__init__(
            prompts, num_steps, cross_replace_steps, self_replace_steps, local_blend
        )
        self.mapper, alphas = seq_aligner.get_refinement_mapper(prompts, tokenizer)
        self.mapper, alphas = self.mapper.to(device), alphas.to(device)
        self.alphas = alphas.reshape(alphas.shape[0], 1, 1, alphas.shape[1])


class AttentionReweight(AttentionControlEdit):
    def replace_cross_attention(self, attn_base, att_replace):
        if self.prev_controller is not None:
            attn_base = self.prev_controller.replace_cross_attention(
                attn_base, att_replace
            )
        attn_replace = attn_base[None, :, :, :] * self.equalizer[:, None, None, :]
        return attn_replace

    def __init__(
        self,
        prompts,
        num_steps: int,
        cross_replace_steps: float,
        self_replace_steps: float,
        equalizer,
        local_blend: Optional[LocalBlend] = None,
        controller: Optional[AttentionControlEdit] = None,
    ):
        super(AttentionReweight, self).__init__(
            prompts, num_steps, cross_replace_steps, self_replace_steps, local_blend
        )
        self.equalizer = equalizer.to(device)
        self.prev_controller = controller


def get_equalizer(
    text: str,
    word_select: Union[int, Tuple[int, ...]],
    values: Union[List[float], Tuple[float, ...]],
):
    if type(word_select) is int or type(word_select) is str:
        word_select = (word_select,)
    equalizer = torch.ones(1, 77)

    for word, val in zip(word_select, values):
        inds = ptp_utils.get_word_inds(text, word, tokenizer)
        equalizer[:, inds] = val
    return equalizer


def aggregate_attention(
    attention_store: AttentionStore,
    res: int,
    from_where: List[str],
    is_cross: bool,
    select: int,
):
    out = []
    attention_maps = attention_store.get_average_attention()
    num_pixels = res**2
    for location in from_where:
        for item in attention_maps[f"{location}_{'cross' if is_cross else 'self'}"]:
            if item.shape[1] == num_pixels:
                cross_maps = item.reshape(len(prompts), -1, res, res, item.shape[-1])[
                    select
                ]
                out.append(cross_maps)
    out = torch.cat(out, dim=0)
    out = out.sum(0) / out.shape[0]
    return out.cpu()


def make_controller(
    prompts: List[str],
    is_replace_controller: bool,
    cross_replace_steps: Dict[str, float],
    self_replace_steps: float,
    blend_words=None,
    equilizer_params=None,
) -> AttentionControlEdit:
    if blend_words is None:
        lb = None
    else:
        lb = LocalBlend(prompts, blend_word)
    if is_replace_controller:
        controller = AttentionReplace(
            prompts,
            NUM_DIFFUSION_STEPS,
            cross_replace_steps=cross_replace_steps,
            self_replace_steps=self_replace_steps,
            local_blend=lb,
        )
    else:
        controller = AttentionRefine(
            prompts,
            NUM_DIFFUSION_STEPS,
            cross_replace_steps=cross_replace_steps,
            self_replace_steps=self_replace_steps,
            local_blend=lb,
        )
    if equilizer_params is not None:
        eq = get_equalizer(
            prompts[1], equilizer_params["words"], equilizer_params["values"]
        )
        controller = AttentionReweight(
            prompts,
            NUM_DIFFUSION_STEPS,
            cross_replace_steps=cross_replace_steps,
            self_replace_steps=self_replace_steps,
            equalizer=eq,
            local_blend=lb,
            controller=controller,
        )
    return controller


def show_cross_attention(
    attention_store: AttentionStore, res: int, from_where: List[str], select: int = 0
):
    tokens = tokenizer.encode(prompts[select])
    decoder = tokenizer.decode
    attention_maps = aggregate_attention(attention_store, res, from_where, True, select)
    images = []
    for i in range(len(tokens)):
        image = attention_maps[:, :, i]
        image = 255 * image / image.max()
        image = image.unsqueeze(-1).expand(*image.shape, 3)
        image = image.numpy().astype(np.uint8)
        image = np.array(Image.fromarray(image).resize((256, 256)))
        image = ptp_utils.text_under_image(image, decoder(int(tokens[i])))
        images.append(image)
    ptp_utils.view_images(np.stack(images, axis=0))


def show_self_attention_comp(
    attention_store: AttentionStore,
    res: int,
    from_where: List[str],
    max_com=10,
    select: int = 0,
):
    attention_maps = (
        aggregate_attention(attention_store, res, from_where, False, select)
        .numpy()
        .reshape((res**2, res**2))
    )
    u, s, vh = np.linalg.svd(
        attention_maps - np.mean(attention_maps, axis=1, keepdims=True)
    )
    images = []
    for i in range(max_com):
        image = vh[i].reshape(res, res)
        image = image - image.min()
        image = 255 * image / image.max()
        image = np.repeat(np.expand_dims(image, axis=2), 3, axis=2).astype(np.uint8)
        image = Image.fromarray(image).resize((256, 256))
        image = np.array(image)
        images.append(image)
    ptp_utils.view_images(np.concatenate(images, axis=1))

In [7]:
def load_512(image_or_path):
    """input PIL Image or image path, resize to 512x512"""
    if isinstance(image_or_path, str):
        image = np.array(Image.open(image_or_path))[:, :, :3]
    elif isinstance(image_or_path, Image.Image):
        image = np.array(image_or_path.convert("RGB"))
    else:
        raise TypeError
    image = np.array(Image.fromarray(image).resize((512, 512)))
    return image

## ERDDCI Inversion


In [8]:
class ERDDCIInversion:
    def __init__(self, model):
        self.model = model
        self.tokenizer = self.model.tokenizer
        self.scheduler = self.model.scheduler
        self.scheduler.set_timesteps(NUM_DIFFUSION_STEPS)
        self.prompt = None
        self.context = None

    @torch.no_grad()
    def latent2image(self, latents, return_type="np"):
        latents = 1 / 0.18215 * latents.detach()
        image = self.model.vae.decode(latents)["sample"]
        if return_type == "np":
            image = (image / 2 + 0.5).clamp(0, 1)
            image = image.cpu().permute(0, 2, 3, 1).numpy()[0]
            image = (image * 255).astype(np.uint8)
        return image

    @torch.no_grad()
    def image2latent(self, image):
        with torch.no_grad():
            if type(image) is Image:
                image = np.array(image)
            if type(image) is torch.Tensor and image.dim() == 4:
                latents = image
            else:
                image = torch.from_numpy(image).float() / 127.5 - 1
                image = image.permute(2, 0, 1).unsqueeze(0).to(device)
                latents = self.model.vae.encode(image)["latent_dist"].mean
                latents = latents * 0.18215
        return latents

    @torch.no_grad()
    def init_prompt(self, prompt: str):
        uncond_input = self.model.tokenizer(
            [""],
            padding="max_length",
            max_length=self.model.tokenizer.model_max_length,
            return_tensors="pt",
        )
        uncond_embeddings = self.model.text_encoder(
            uncond_input.input_ids.to(self.model.device)
        )[0]
        text_input = self.model.tokenizer(
            [prompt],
            padding="max_length",
            max_length=self.model.tokenizer.model_max_length,
            truncation=True,
            return_tensors="pt",
        )
        text_embeddings = self.model.text_encoder(
            text_input.input_ids.to(self.model.device)
        )[0]
        self.context = torch.cat([uncond_embeddings, text_embeddings])
        self.prompt = prompt

    @torch.no_grad()
    def prev_step(
        self,
        model_output: Union[torch.FloatTensor, np.ndarray],
        timestep: int,
        sample: Union[torch.FloatTensor, np.ndarray],
    ):
        prev_timestep = (
            timestep
            - self.scheduler.config.num_train_timesteps
            // self.scheduler.num_inference_steps
        )
        alpha_prod_t = self.scheduler.alphas_cumprod[timestep]
        alpha_prod_t_prev = (
            self.scheduler.alphas_cumprod[prev_timestep]
            if prev_timestep >= 0
            else self.scheduler.final_alpha_cumprod
        )
        beta_prod_t = 1 - alpha_prod_t
        pred_original_sample = (
            sample - beta_prod_t**0.5 * model_output
        ) / alpha_prod_t**0.5
        pred_sample_direction = (1 - alpha_prod_t_prev) ** 0.5 * model_output
        prev_sample = (
            alpha_prod_t_prev**0.5 * pred_original_sample + pred_sample_direction
        )
        return prev_sample

    @torch.no_grad()
    def next_step(
        self,
        model_output: Union[torch.FloatTensor, np.ndarray],
        timestep: int,
        sample: Union[torch.FloatTensor, np.ndarray],
    ):
        timestep, next_timestep = (
            min(
                timestep
                - self.scheduler.config.num_train_timesteps
                // self.scheduler.num_inference_steps,
                999,
            ),
            timestep,
        )
        alpha_prod_t = (
            self.scheduler.alphas_cumprod[timestep]
            if timestep >= 0
            else self.scheduler.final_alpha_cumprod
        )
        alpha_prod_t_next = self.scheduler.alphas_cumprod[next_timestep]
        beta_prod_t = 1 - alpha_prod_t
        next_original_sample = (
            sample - beta_prod_t**0.5 * model_output
        ) / alpha_prod_t**0.5
        next_sample_direction = (1 - alpha_prod_t_next) ** 0.5 * model_output
        next_sample = (
            alpha_prod_t_next**0.5 * next_original_sample + next_sample_direction
        )
        return next_sample

    @torch.no_grad()
    def get_noise_pred_invert(self, latents, t, context):
        noise_pred = self.model.unet(latents, t, encoder_hidden_states=context)[
            "sample"
        ]
        return noise_pred

    @torch.no_grad()
    def get_noise_pred(
        self, latents, t, guidance_scale, context=None, low_resource=LOW_RESOURCE
    ):
        if context is None:
            context = self.context
        uncond_embeddings, text_embeddings = context.chunk(2)
        if low_resource:
            noise_pred_uncond = self.model.unet(
                latents, t, encoder_hidden_states=uncond_embeddings
            )["sample"]
            noise_pred_cond = self.model.unet(
                latents, t, encoder_hidden_states=text_embeddings
            )["sample"]
        else:
            latents_input = torch.cat([latents] * 2)
            noise_pred = self.model.unet(
                latents_input, t, encoder_hidden_states=context
            )["sample"]
            noise_pred_uncond, noise_pred_cond = noise_pred.chunk(2)

        noise_pred = noise_pred_uncond + guidance_scale * (
            noise_pred_cond - noise_pred_uncond
        )

        return noise_pred

    @torch.no_grad()
    def ddim_loop(self, latent):
        aux_latent_list = [latent.clone().detach()]
        ori_latent_list = [latent.clone().detach()]
        _, text_embeddings = self.context.chunk(2)
        for i in range(NUM_DIFFUSION_STEPS):
            t = self.model.scheduler.timesteps[
                len(self.model.scheduler.timesteps) - i - 1
            ]
            ori_latent_t = ori_latent_list[-1]
            # print(f"[Debug] ddim_loop: {i=}\t{t=}")
            noise_pred = self.get_noise_pred_invert(ori_latent_t, t, text_embeddings)
            ori_latent_t_next = self.next_step(noise_pred, t, ori_latent_t)
            ori_latent_list.append(ori_latent_t_next)

            aux_latent_t = aux_latent_list[-1]
            noise_pred = self.get_noise_pred_invert(
                ori_latent_t_next, t, text_embeddings
            )
            aux_latent_t_next = self.next_step(noise_pred, t, aux_latent_t)
            aux_latent_list.append(aux_latent_t_next)

        return ori_latent_list, aux_latent_list

    @torch.no_grad()
    def ddim_inversion(self, image):
        latent = self.image2latent(image)
        image_rec = self.latent2image(latent)
        ori_latent_list, aux_latent_list = self.ddim_loop(latent)
        return image_rec, ori_latent_list, aux_latent_list

    @torch.no_grad()
    def invert(self, image_path, prompt: str, verbose=False):
        self.init_prompt(prompt)
        ptp_utils.register_attention_control(self.model, None)
        image_gt = load_512(image_path)
        if verbose:
            print("ERDDCI inversion...")

        image_rec, ori_latent_list, aux_latent_list = self.ddim_inversion(image_gt)
        return (image_gt, image_rec), ori_latent_list, aux_latent_list

    @torch.no_grad()
    def get_encode_invert_inference_latent(
        self, image_or_path, prompt: str, guidance_scale=1.0
    ):
        self.init_prompt(prompt)
        ptp_utils.register_attention_control(self.model, None)
        image_gt = load_512(image_or_path)
        encode_latent = self.image2latent(image_gt)

        ori_latent_list, aux_latent_list = self.ddim_loop(encode_latent)
        ddim_invert_latent = ori_latent_list[-1]
        erddci_invert_latent = aux_latent_list[-1]

        ddim_inference_latent = ddim_invert_latent.clone().detach()
        erddci_inference_latent = erddci_invert_latent.clone().detach()
        reversed_ori_latent_list = list(reversed(ori_latent_list))
        for i in range(NUM_DIFFUSION_STEPS):
            t = self.model.scheduler.timesteps[i]
            ori_latent_t = reversed_ori_latent_list[i]
            # print(f"[Debug] inference: {i=}\t{t=}")
            noise_pred = self.get_noise_pred(
                ori_latent_t, t, guidance_scale, self.context
            )

            ddim_inference_latent = self.prev_step(noise_pred, t, ddim_inference_latent)

            erddci_inference_latent = self.prev_step(
                noise_pred, t, erddci_inference_latent
            )

        return (
            encode_latent,
            ddim_invert_latent,
            erddci_invert_latent,
            ddim_inference_latent,
            erddci_inference_latent,
        )

In [9]:
erddci_inversion = ERDDCIInversion(ldm_stable)

## Reconstruct Evaluation


In [10]:
import os
from datasets import load_dataset


def calculate_metrics(
    original_image, reconstructed_image, LPIPS=lpips.LPIPS(net="vgg").to(device)
):
    """
    Calculate LPIPS, PSNR, and SSIM Metrics
    """

    # Convert to PyTorch Tensor and Normalize to [-1, 1]
    def to_tensor(img):
        img_t = torch.tensor(img).float().permute(2, 0, 1) / 127.5 - 1
        return img_t.unsqueeze(0).to(device)

    original_tensor = to_tensor(original_image)
    reconstructed_tensor = to_tensor(reconstructed_image)

    lpips_value = LPIPS(original_tensor, reconstructed_tensor).item()

    psnr_value = PSNR(original_image, reconstructed_image)

    ssim_value = SSIM(
        original_image, reconstructed_image, channel_axis=-1, data_range=255
    )

    return lpips_value, psnr_value, ssim_value

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/home/djm/ERDDCI/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/djm/ERDDCI/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/djm/ERDDCI/.venv/lib/python3.11/site-packages/lpips/weights/v0.1/vgg.pth


Update the `image_folder` path as follows:

- For the `wild-ti2i` dataset, use `real_path/to/wild-ti2i`
- For the `imagenetr-ti2i` dataset, use `real_path/to/imagenetr-ti2i`

For **DOCCI**, simply provide an empty directory named `DOCCI` – the actual image data will be automatically cached by the `datasets` package.

The restructured results will also be saved in their corresponding folders.


In [11]:
image_folder = "./ReconDataset/imagenetr-ti2i/"
output_folder = os.path.join(
    image_folder, f"recon_G{int(GUIDANCE_SCALE)}S{NUM_DIFFUSION_STEPS}"
)
results_file = os.path.join(
    output_folder, f"erddci_results_G{int(GUIDANCE_SCALE)}S{NUM_DIFFUSION_STEPS}.txt"
)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Creating output folder: {output_folder}")

if "DOCCI" in image_folder:
    recon_test_dataset = load_dataset("google/docci", split="test")
else:
    recon_test_dataset = load_dataset(
        image_folder, split="test", trust_remote_code=True
    )

print(recon_test_dataset)
print(recon_test_dataset.info.dataset_name)
print(recon_test_dataset[0])

Creating output folder: ./ReconDataset/imagenetr-ti2i/recon_G1S50
Dataset({
    features: ['image', 'description'],
    num_rows: 30
})
imagenetr-ti2i
{'image': '/home/djm/ERDDCI/ReconDataset/imagenetr-ti2i/data/a cartoon of a castle.jpg', 'description': 'a cartoon of a castle'}


In [13]:
results_data = []
RECONSTRUCT_NUM_LIMIT = 500  # limit max reconstruct num
loop_counter = 0
with open(results_file, "w") as f:
    f.write(
        f"Reconstruction Test Results (Input Folder: {image_folder}, Output Folder: {output_folder}):\n"
    )

    for item in recon_test_dataset:
        if loop_counter >= RECONSTRUCT_NUM_LIMIT:
            break

        image_or_path = item["image"]
        prompt = item["description"]
        if recon_test_dataset.info.dataset_name == "docci":
            filename = f"{item['example_id']}.png"
        else:
            filename = os.path.basename(image_or_path)

        print(f"Processing: {filename}, {prompt=}")
        f.write(f"Processing image: {filename}\n")

        # Load image
        original_image = load_512(image_or_path)
        if original_image is None:
            continue

        try:
            # invert and inference
            (
                encode_latent,
                ddim_invert_latent,
                erddci_invert_latent,
                ddim_inference_latent,
                erddci_inference_latent,
            ) = erddci_inversion.get_encode_invert_inference_latent(
                image_or_path, prompt, guidance_scale=GUIDANCE_SCALE
            )

            # Decode
            recon_image_encode = erddci_inversion.latent2image(encode_latent)
            recon_image_ddim = erddci_inversion.latent2image(ddim_inference_latent)
            recon_image_erddci = erddci_inversion.latent2image(erddci_inference_latent)
            # Calculate evaluation metrics
            metrics_ddim = calculate_metrics(recon_image_encode, recon_image_ddim)
            metrics_erddci = calculate_metrics(recon_image_encode, recon_image_erddci)

            # Save result
            results_data.append(
                {
                    "filename": filename,
                    "ddim_lpips": metrics_ddim[0],
                    "ddim_psnr": metrics_ddim[1],
                    "ddim_ssim": metrics_ddim[2],
                    "erddci_lpips": metrics_erddci[0],
                    "erddci_psnr": metrics_erddci[1],
                    "erddci_ssim": metrics_erddci[2],
                }
            )
            print(
                f"{filename} - DDIM: LPIPS={metrics_ddim[0]:.4f}, PSNR={metrics_ddim[1]:.4f}, SSIM={metrics_ddim[2]:.4f}"
            )
            print(
                f"{filename} - ERDDCI: LPIPS={metrics_erddci[0]:.4f}, PSNR={metrics_erddci[1]:.4f}, SSIM={metrics_erddci[2]:.4f}"
            )
            # Save recon images
            output_image_encode = Image.fromarray(recon_image_encode)
            output_path_encode = os.path.join(output_folder, f"recon_encode_{filename}")
            output_image_encode.save(output_path_encode)
            print(f"  Reconstructed (GT) saved to: {output_path_encode}")

            output_image_ddim = Image.fromarray(recon_image_ddim)
            output_path_ddim = os.path.join(output_folder, f"recon_ddim_{filename}")
            output_image_ddim.save(output_path_ddim)
            print(f"  Reconstructed (DDIM) saved to: {output_path_ddim}")

            output_image_erddci = Image.fromarray(recon_image_erddci)
            output_path_erddci = os.path.join(output_folder, f"recon_erddci_{filename}")
            output_image_erddci.save(output_path_erddci)
            print(f"  Reconstructed (ERDDCI) saved to: {output_path_erddci}")

            loop_counter += 1
        except Exception as e:
            print(f"  Error processing {filename}: {e}")

    df_results = pd.DataFrame(results_data)
    average_metrics = df_results[
        [
            "ddim_lpips",
            "ddim_psnr",
            "ddim_ssim",
            "erddci_lpips",
            "erddci_psnr",
            "erddci_ssim",
        ]
    ].mean()
    print("\nAverage Metrics:")
    print(average_metrics)

    # 将 DataFrame 保存为 CSV 文件
    csv_output_file = results_file.replace(".txt", ".csv")
    df_results.to_csv(csv_output_file, index=False)
    print(f"\nResults saved to {csv_output_file}")

    json_output_file = results_file.replace(".txt", ".json")
    df_results.to_json(json_output_file, orient="records", indent=4)
    print(f"Results saved to {json_output_file}")

print(
    "Reconstruction test completed! Results are saved in reconstruction_results.txt, and reconstructed images are in the corresponding '_recon' folder."
)

Processing: a cartoon of a castle.jpg, prompt='a cartoon of a castle'
a cartoon of a castle.jpg - DDIM: LPIPS=0.0443, PSNR=34.1093, SSIM=0.9735
a cartoon of a castle.jpg - ERDDCI: LPIPS=0.0001, PSNR=69.5340, SSIM=1.0000
  Reconstructed (GT) saved to: ./ReconDataset/imagenetr-ti2i/recon_G1S50/recon_encode_a cartoon of a castle.jpg
  Reconstructed (DDIM) saved to: ./ReconDataset/imagenetr-ti2i/recon_G1S50/recon_ddim_a cartoon of a castle.jpg
  Reconstructed (ERDDCI) saved to: ./ReconDataset/imagenetr-ti2i/recon_G1S50/recon_erddci_a cartoon of a castle.jpg
Processing: a cartoon of a cat.jpg, prompt='a cartoon of a cat'
a cartoon of a cat.jpg - DDIM: LPIPS=0.0518, PSNR=29.8792, SSIM=0.9384
a cartoon of a cat.jpg - ERDDCI: LPIPS=0.0000, PSNR=65.7139, SSIM=1.0000
  Reconstructed (GT) saved to: ./ReconDataset/imagenetr-ti2i/recon_G1S50/recon_encode_a cartoon of a cat.jpg
  Reconstructed (DDIM) saved to: ./ReconDataset/imagenetr-ti2i/recon_G1S50/recon_ddim_a cartoon of a cat.jpg
  Reconstructe

KeyboardInterrupt: 